<a href="https://colab.research.google.com/github/yuhi-sa/UT/blob/main/Unscented_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unscented Transformation


# はじめに
Unscented Transformation(アンセンテッド変換,U変換)は，Unscented Kalman Filterの中で出てくるけど，U変換単体での説明がなくて困ってるので今の理解をまとめました．
間違ってる所など教えていただけるととてもありがたいです．

# Unscented Transformationの概要
U変換は，標準正規分布に従う確率変数$x$の平均$\bar{x}$と分散共分散行列$P_x$が既知であるとき，$x$の非線形変換$y=f(x)$で変換される確率変数$y$の$\bar{y}$と$P_y$を推定する方法である．

まず，この場合，モンテカロ的に
$\bar{y}\simeq\frac{1}{N}\sum_{i=1}^Nf(x_i)$  
$P_y \simeq \frac{1}{N}\sum_{i=1}^N(f(x_i)-\bar{y})(f(x_i)-\bar{y})^T$  
のように計算することが思いつくが，精度をよく計算するにはNを大きくする必要があり，実用上問題がある．

線形近似をすることなく，モンテカロ法のよいところを利用できるように，できるだけ少ないサンプル点を用いて，変換後の確率変数の統計的性質を推定する方法がU変換である．

まず，確率変数$x$からサンプルする値(シグマ点)を決め，シグマ点を非線形変換し，変換した値から$y$の$\bar{y}$と$P_y$を求める．

参考1：[UKF （Unscented Kalman Filter）っ て何 ？](https://www.jstage.jst.go.jp/article/isciesci/50/7/50_KJ00004329717/_pdf)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import scipy.linalg

入力には$X=(X_1,X_2)$を用いる．  

平均ベクトルは，$\mu=(E[X_1],E[X_2])=(\bar{x}_1,\bar{x}_2)$  
分散共分散ベクトルは，$P_x = [
    \begin{array}{cc}
      var[X_1] & cov[X_1,X_2] \\
      cov[X_2,X_1] & var[X_2]
    \end{array}
    ]
    = [
    \begin{array}{cc}
      \sigma_1^2 & \sigma_1\sigma_2 \\
      \sigma_1\sigma_2 & \sigma_2^2
    \end{array}
    ]
$  
で表すことができる．  
よって，  $X_1,X_2$にそれぞれ平均0分散1，平均1分散2の標準正規分布を用いると考えると，  
$\bar{x}=[0, 1]$  
$P_x = [
    \begin{array}{cc}
      1 & 2 \\
      2 & 4
    \end{array}
    ]
$
となる．

これを入力として用いる．出力は1次元とし$f(x)=x[0]*x[1]$の非線形変換を考える．

In [ ]:
# xの次元数
n = 2
# yの次元数
m = 1

# xの平均と分散
x_mean = np.array([0, 1])
x_P = np.array([[1,2],[2,4]])
print("xの平均",x_mean)
print("xの分散共分散行列",x_P)

# xの非線形変換
def f(x):
  return [x[0]*x[1]]

xの平均 [0 1]
xの分散共分散行列 [[1 2]
 [2 4]]


# シグマ点を計算する
$\sigma_0 = \bar{x} \tag{1}$  
$\sigma_i = \bar{x}+(\sqrt{(n+\lambda)}P_x) \tag{2}$  
$\sigma_i = \bar{x}-(\sqrt{(n+\lambda)}P_x) \tag{3}$ 

ここで，$\lambda$は以下のように計算される．  
$\lambda = \alpha^2*(n+\kappa)-n \tag{4}$

ここで，$\alpha$と$\kappa$はハイパーパラメータである．  
  
$\alpha$:平均の状態値の周りのシグマ ポイントの広がりを決定します。0-1のスカラー値として指定します．これは通常、小さい正の数値です．シグマポイントの広がりは$\alpha$に比例します．値が小さいほど、シグマポイントは平均の状態に近くなります．  

$\kappa$：通常は0に設定されます．値が小さいほど，シグマポイントは平均の状態に近くなります．広がりは$\kappa$の平方根に比例します。  

参考1：[Unscented Kalman Filter， MathWorks](https://jp.mathworks.com/help/control/ref/ukf_block.html)  
参考2：[Unscentedカルマンフィルタを使用した自己位置推定MATLAB, Pythonサンプルプログラム](https://myenigma.hatenablog.com/entry/20140614/1402731732)


In [ ]:
# パラメータ
alpha = 0.5
kappa = 0
# 式4
lambd=alpha**2*(n+kappa)-n
print("ラムダ", lambd)
# シグマ点をサンプリング
sigma = np.zeros((n, 2*n+1))

# 式1
sigma[:,0] = x_mean
gamma=math.sqrt(n+lamda)
# 式2
for i in range(n):
    sigma[:,i+1] = x_mean + gamma * x_P[:,i]
# 式3
for i in range(n):
    sigma[:,i+n+1] =x_mean - gamma * x_P[:,i]

print("選択されたシグマ点")
for i in range(2*n+1):
  print(sigma[:,i])

ラムダ -1.5
選択されたシグマ点
[0. 1.]
[0.00141421 1.00282843]
[0.00282843 1.00565685]
[-0.00141421  0.99717157]
[-0.00282843  0.99434315]


# 変換
非線形変換により$y_{\sigma}$をもとめる．  
$y_{\sigma}=f(\sigma) \tag{5}$  

重み関数$w_i$を求める．  
$w_0=\frac{\lambda}{n+\lambda} \tag{6}$  
$w_i=\frac{1}{2(n+\lambda)} \tag{7}$

$y$の平均と分散は以下のように計算できる．  
$\bar{y} \simeq \sum_{i=0}^{2n}w_iy_{\sigma i} \tag{8}$  
$P_y \simeq \sum_{i=0}^{2n} w_i (y_{\sigma i}-\bar{y})*(y_{\sigma i}-\bar{y})^T \tag{9}$




In [ ]:
sigma_y = np.zeros((m, 2*n+1))
# 非線形変換　式5
for i in range(len(sigma)):
  sigma_y[:,i] = f(sigma[:,i])

w = np.zeros((1, 2*n+1))
# 重み関数計算　式6,7
w[:,0] = lambd/(n+lambd)
for i in range(2*n):
  w[:,i+1] = 1/(2*(n+lambd))
print("重み関数", w)
y_mean=0
# yの平均計算 式8
for i in range(2*n+1):
  y_mean += w[:,i]*sigma_y[:,i]

y_P = 0
# yの分散共分散計算 式9
for i in range(2*n+1):
  y_P += w[:,i]*(sigma_y[:,i]-y_mean)*(sigma_y[:,i]-y_mean).T


print("yの平均",y_mean)
print("yの分散共分散行列",y_P)

重み関数 [[-3.  1.  1.  1.  1.]]
yの平均 [0.00141821]
yの分散共分散行列 [-8.47032947e-22]


# サンプリングで計算した結果と比較

In [ ]:
def samplingY(samp):
  y = np.zeros((m,samp))
  for i in range(samp):
    x = [random.random()*2-1, random.random()*4-2]
    y[:,i]=f(val)
    print(x)
  print("平均の差",abs(np.mean(y)))
  print("yの分散共分散行列",abs(np.var(y)))
  print("-----------------")

# サンプル数を少しずつ増やす
samplingY(10)
samplingY(100)
samplingY(1000)
samplingY(10000)
samplingY(1000000)

Streaming output truncated to the last 5000 lines.
[0.4447364884142575, 1.2347813795639455]
[-0.9112610520711433, 1.7878587249996056]
[-0.9389730041649058, -1.6674371343524337]
[0.6009386486272992, 1.620192255591975]
[-0.8642769097363316, 0.24262992463932243]
[-0.4589353234417748, -1.4231318810742657]
[-0.6041921527822163, -0.22044987885834022]
[0.974130836836895, -1.4817029713129846]
[-0.6750086070054253, 0.9515166804859589]
[-0.02640940564813743, -1.3093842351084364]
[-0.33144784026910346, 0.9845645543396526]
[-0.6197166392519542, 1.9022039317127653]
[-0.9833771520231946, -0.20100233443743587]
[-0.5343384363202366, -1.409192881507519]
[0.6947051418562107, 1.2409686880635427]
[-0.5544734496646573, -0.041531269540264315]
[-0.26532409025893, -1.1772526170432878]
[0.8413732774339107, 1.282893846964447]
[0.3881790350220562, -1.2315429389830763]
[0.9508378459625835, 1.6560432792193254]
[-0.6451233492862609, -1.5829811901166875]
[-0.06877662543017227, -1.6015127435768184]
[0.194279465763607

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
